In [17]:
import math
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense ,LSTM
import matplotlib.pyplot as plt
import math
from numpy.random import seed
seed(3)
import tensorflow as tf
tf.compat.v1.set_random_seed(2)

In [18]:
df1=pd.read_csv("/content/drive/MyDrive/States/Code and Data/Covid_Data.csv")

In [19]:
Epochs=np.linspace(25,35,11,dtype=int)
Nodes=np.linspace(25,35,11,dtype=int)
Batch=5
no=9

In [20]:
p="/content/drive/MyDrive/States/NEW_MODEL/ANDHRA_PRADESH"

In [21]:
df=df1[df1["Region"]=="Andhra Pradesh"]

In [22]:
df

,S. No.,Date,Region,Confirmed Cases,Active Cases,Cured/Discharged,Death
1329,1330,01/01/1970,Andhra Pradesh,2074868,1912.0,2058490.0,14466.0
1330,1331,21/12/2014,Andhra Pradesh,2074976,1878.0,2058631.0,14467.0
1331,1332,21/12/2015,Andhra Pradesh,2075108,1823.0,2058817.0,14468.0
1332,1333,12/03/2020,Andhra Pradesh,1,1.0,0.0,0.0
1333,1334,13/03/2020,Andhra Pradesh,1,1.0,0.0,0.0
...,...,...,...,...,...,...,...
1996,1997,28/01/2022,Andhra Pradesh,2236047,109493.0,2111975.0,14579.0
1997,1998,29/01/2022,Andhra Pradesh,2248608,113300.0,2120717.0,14591.0
1998,1999,30/01/2022,Andhra Pradesh,2260181,115425.0,2130162.0,14594.0
1999,2000,31/01/2022,Andhra Pradesh,2270491,116031.0,2139854.0,14606.0


In [23]:
data=df.filter(["Active Cases"])
dataset=data[1:].values
training_data_len=math.ceil(len(dataset)*.85)

In [24]:
scaler=MinMaxScaler(feature_range=(0,1))
scaled_data=scaler.fit_transform(dataset)
l=60

train_data=scaled_data[0:training_data_len,:]
x_train=[]
y_train=[]

for i in range(l,len(train_data)):
  x_train.append(train_data[i-l:i,0])
  y_train.append(train_data[i,0])

x_train,y_train=np.array(x_train),np.array(y_train)
x_train=np.reshape(x_train,(len(x_train),len(x_train[0]),1))



In [25]:
for N in Nodes:
  for E in Epochs:
    if N!=0:
      model=Sequential()
      model.add(LSTM(N,return_sequences=False,input_shape=(len(x_train[0]),1)))
      # model.add(LSTM(10,return_sequences=False))
      model.add(Dense(12))
      model.add(Dense(1))
      model.compile(optimizer='adam',loss='mean_squared_error',metrics=['accuracy']) #mean_squared_error
      hist=model.fit(x_train,y_train,batch_size=Batch,epochs=E) #upto 50 epoch

      loss=hist.history["loss"]

      Testsss=p+"/Test/"+str(N)+"/"+str(E)+".xlsx"
      lossss=p+"/Train/"+str(N)+"/"+str(E)+".xlsx"
      Predsss=p+"/Pred/"+str(N)+"/"+str(E)+".xlsx"
      dict1={"Epochs":np.linspace(1,E,E),"Loss":loss}
      df_3=pd.DataFrame(dict1)
      xl=pd.ExcelWriter(lossss)
      df_3.to_excel(xl)
      xl.save()

      #test
      test_data=scaled_data[training_data_len-l:,:]
      x_test=[]
      y_test=dataset[training_data_len:,:]

      for i in range(l,len(test_data)):
        x_test.append(test_data[i-l:i,0])

      #convert data to numpy
      x_test=np.array(x_test)

      #reshape
      x_test=np.reshape(x_test,(len(x_test),len(x_test[0]),1))

      predictions=model.predict(x_test)
      Test_Accuracy=model.evaluate(x_test,predictions)
      predictions=scaler.inverse_transform(predictions)


      predictions1=np.reshape(predictions,(len(predictions)))

      dict1={"Days":np.linspace(len(x_train)+1,len(x_train)+len(x_test),len(x_test)),"Test Prediction":predictions1}
      df_3=pd.DataFrame(dict1)
      xl=pd.ExcelWriter(Testsss)
      df_3.to_excel(xl)
      xl.save()

      xx=np.reshape(x_test[-1],(1,l,1))
      prediction=model.predict(xx)
      pred=np.array([])
      for i in range(300):
        prediction=model.predict(xx)
        if prediction[0]<0:
          prediction[0]=xx[0][-1]
        pred=np.append(pred,prediction[0],axis=0)
        xx=np.delete(xx,0,axis=1)
        xx=np.append(xx[0],prediction,axis=0)
        xx=np.reshape(xx,(1,l,1))

      if len(pred)==0:
        continue

      pred=np.reshape(pred,(len(pred),1))
      pred=scaler.inverse_transform(pred)

      XX=np.linspace(len(dataset),len(dataset)+pred.size-1,pred.size)

      pred1=np.reshape(pred,(len(pred)))
      dict1={"Days":XX,"Third Wave":pred1}
      df_3=pd.DataFrame(dict1)
      xl=pd.ExcelWriter(Predsss)
      df_3.to_excel(xl)
      xl.save()
      print(N,E)


    

            


Epoch 1/25
103/103 [==============================] - 5s 21ms/step - loss: 0.0079 - accuracy: 0.0020
Epoch 2/25
103/103 [==============================] - 3s 24ms/step - loss: 8.6019e-04 - accuracy: 0.0020
Epoch 3/25
103/103 [==============================] - 4s 43ms/step - loss: 3.8112e-04 - accuracy: 0.0020
Epoch 4/25
103/103 [==============================] - 3s 29ms/step - loss: 3.6151e-04 - accuracy: 0.0020
Epoch 5/25
103/103 [==============================] - 2s 22ms/step - loss: 2.6506e-04 - accuracy: 0.0020
Epoch 6/25
103/103 [==============================] - 2s 22ms/step - loss: 2.6464e-04 - accuracy: 0.0020
Epoch 7/25
103/103 [==============================] - 2s 21ms/step - loss: 2.5164e-04 - accuracy: 0.0020
Epoch 8/25
103/103 [==============================] - 2s 21ms/step - loss: 2.2754e-04 - accuracy: 0.0020
Epoch 9/25
103/103 [==============================] - 2s 21ms/step - loss: 2.8425e-04 - accuracy: 0.0020
Epoch 10/25
103/103 [==============================] - 2s 2

FileNotFoundError: ignored